<a href="https://colab.research.google.com/github/Fjoru/TM10007_PROJECT/blob/Jessica/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment Prediction of tumor grade in brain cancer
By Jessica Barends, Gonnie van Erp, Erik Kemper en Carlijn Oerlemans

In [0]:
# Run install for use in colab environment
!pip install --upgrade pip
!pip install -q --upgrade git+https://github.com/Fjoru/TM10007_PROJECT
!pip install ipdb -q
!pip install seaborn
!pip install tensorflow

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)


## Import section


In [0]:
import ipdb
import pandas as pd
import seaborn as sns
import numpy as np
# import tensorflow as tf

# Classifiers
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

## preprocessing

In [0]:
def Replace(i):
    if isinstance(i, int):
          return i
    try:
        float(i)
        return float(i)
    except:
        return np.nan

def preprocessing_steps(X_design, Y_design):
    # remove strings from data
    X_design = X_design.applymap(func=Replace)

    # set 0.0 as NaN
    X_design = X_design.replace(0, np.nan)

    # set Inf as NaN
    pd.options.mode.use_inf_as_na = True # inf waardes zien als nan

    # remove features with less than 60% values
    X_design = X_design.dropna(thresh=round(len(X_design)*0.8), axis='columns')

    # remove sample with less than 60% values
    # join features and labels
    XY_design = X_design.join(Y_design)
    # remove samples with not enough values
    XY_design = XY_design.dropna(thresh=round(XY_design.shape[1]*0.6))

    # remove samples without a label
    XY_design['labels'].replace(np.nan, '', inplace=True)
    XY_design = XY_design[XY_design['labels'].astype(bool)]
    
    # Add missing value's 
    X_design = X_design.fillna(0)

    # normalisation of values
    X_design = pd.DataFrame(preprocessing.normalize(X_design.T, norm='l2'), columns=X_design.index, index=X_design.columns )
    X_design = X_design.T

    return X_design, Y_design

## Run Pipeline

run all predefined steps

In [0]:
# Data loading functions.
from brats.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
data = pd.DataFrame(data)

# split labels and values
data_X = data.drop(columns=['label'])
data_Y = data[['label']]

# data split index forming
Test_split = model_selection.StratifiedKFold(n_splits=10)

for design_index, test_index in Test_split.split(data_X, data_Y):
    
    X_design = data_X.iloc[design_index]
    Y_design = data_Y.iloc[design_index]
    
    X_test = data_X.iloc[test_index]
    Y_test = data_Y.iloc[test_index]

    # run preprocessing step
    X_design, Y_design = preprocessing_steps(X_design, Y_design)

    # save data to csv for manual check
    X_design.to_csv('data_X.csv')
    Y_design.to_csv('data_Y.csv')

The number of samples: 167
The number of columns: 725
